## How to use Pandas more efficiently

The Pandas library is the tool that almost every data scientist will use every day. It is an awesome tool to explore and work with data. 

In this notebook, we would like to share a few good tricks such as using built-in functions and adopting vectorized operations that can speed up our pandas code and improve our productivity. The agenda would be: 

1. Selecting Rows and Columns using **.iloc[]** function is faster

2. To iterate through all of rows of a dataframe, in terms of efficiency, vectorizing over pandas series > **.apply()** > **.iterrows()**

3. Try to use more built-in functions such as **replace()** and **groupby()** functions. It could also speed up the process than our coding from scratch. 

In the following notebook, we will use the following dataset from Kaggle: 


[Crime in Chicago](https://www.kaggle.com/datasets/onlyrohit/crimes-in-chicago?resource=download) 

We took a random sample from it and the csv file is named **crimes_in_Chicago_subset.csv** which contains 36082 data points. 

In [2]:
### if we are using google colab, we need to run this cell to specify the path for data loading
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    # specify the path of the folder containing "file_name" by changing the lecture index:
    lecture_index = '01'
    path_to_file = '/content/gdrive/My Drive/BT5153_2023/codes/lab_lecture{}/'.format(lecture_index) 
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd

In [2]:
import pandas as pd
import numpy as np
df_data = pd.read_csv('../data/crimes_in_Chicago_subset.csv')

In [3]:
df_data.shape

(36082, 22)

### 1. Select rows and columns efficiently

In [4]:
%timeit -r5 -n10 df_data.loc[range(0,600)]

234 µs ± 94.8 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [5]:
%timeit -r5 -n10 df_data.iloc[range(0,600)]

163 µs ± 83.4 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


We can see iloc[] perform much faster than loc[] in selecting rows

In [6]:
sub_cols = ["ID", "Case Number", "Date", "Block"]
%timeit -r5 -n10 df_data.loc[:,sub_cols]

760 µs ± 95.7 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [7]:
first3_cols = df_data.columns[:4]
%timeit -r5 -n10 df_data.iloc[:,:4]

661 µs ± 114 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


We can see also that using the column indexing using .iloc[] is still faster than loc[]. So it is better to use .iloc[] for efficiency. However, sometimes, it would be more convenient to use .loc[] to select certain columns by name directly

### 2. Iterate through all rows of Dataframe

It is quite common to create a new column based on one or more current columns using Pandas. Think about feature engineering. We can see the different ways for this operation using Pandas. The performances would be compared. 

For example, we would like to create a new column which is the sum of square of **X Coordinate** and **Y Coordinate**

In [8]:
sum_square = lambda x, y: (x+y) ** 2
print(sum_square(2,3))

25


In [9]:
test_data = df_data[['X Coordinate', 'Y Coordinate']].copy()

In [10]:
%timeit -r5 -n10 test_data.loc[:,'magic'] = [sum_square(value[0], value[1]) for _, value in test_data.iterrows()]
%timeit -r5 -n10 test_data.loc[:,'magic'] = test_data.apply(lambda row: sum_square(row[0], row[1]), axis=1)
%timeit -r5 -n10 test_data.loc[:,'magic']  = test_data.apply(lambda row: sum_square(row[0], row[1]), raw=True, axis=1)
%timeit -r5 -n10 test_data.loc[:,'magic']  = np.vectorize(sum_square)(test_data.iloc[:,0], test_data.iloc[:,1])
%timeit -r5 -n10 test_data.loc[:,'magic']  = np.power(test_data.iloc[:,0]+test_data.iloc[:,1], 2)
#%timeit -r5 -n10 test_data.loc[:,'magic'] = [sum_square(value[0], value[1]) for _, value in test_data.iterrows()]

441 ms ± 1.45 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)
141 ms ± 3.14 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)
55.2 ms ± 626 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)
4.86 ms ± 108 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)
292 µs ± 55.1 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


The best solution is able to achieve **1700** time speed-up compared to the slowest one  

#### Key Take-away:

1. You should never use iterrows(). If you need to loop through a dataframe, try **itertuples**
2. **raw=True** in the **apply** function is able to  bypass the overhead associated with the Pandas series object. Therefore, it can speed up the apply process.
3. Make the function "vetorized" can yield the highest speed up. 

In most of cases, if the operation can be represented as linear algebra operations on matrices and scalar values, it can be vectorized using Numpy methods. 

For some operations/functions, it might not be easily vectorized. I prefer parallel processing because it requires the least amount of rewrite of your existing code. You simply have to add a few lines wrapper code to make it work. The example below illustrates how you can do this. For example, we can try **[Dask](https://www.dask.org/)**

In [11]:
import dask.dataframe as dd
test_data = df_data[['X Coordinate', 'Y Coordinate']].copy()
ddf = dd.from_pandas(test_data, npartitions=4)
sum_square = lambda row: (row[0]+row[1]) ** 2

In [12]:
%%timeit -r5 -n10 
dask_series = ddf.apply(sum_square, axis=1, meta=('magic','float'))
ddf['magic'] = dask_series
df_new = ddf.compute

1.31 ms ± 175 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


The parallelization is able to have a 2.8 times speed up compared to the apply function in the mode of the serial processing. 

But for parallel processing, there is a difference between CPU bound (heavy scientific computing and data is in memory) vs I/O bound (making API request over internet). For cpu bound, we should use multiprocess while the task is I/O bound, the multithreading one would be better choice. 


### 3. Try to use build-in functions in Pandas

If we would like to replace values in Dataframe, we can call **.replace()** directly. 

In [108]:
%timeit -r5 -n10 df_data['Description'].loc[df_data.Description=='SIMPLE'] = 'simple'

The slowest run took 8.89 times longer than the fastest. This could mean that an intermediate result is being cached.
3.32 ms ± 3.92 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [109]:
%timeit -r5 -n10 df_data['Description'].replace({"simple": "SIMPLE"}, inplace=True)

2.18 ms ± 1.15 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


We can see **.replace()** method is faster than the conventional coding method

In [110]:
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10283110,HY471210,01/01/2011 12:00:00 AM,104XX S WENTWORTH AVE,1753,OFFENSE INVOLVING CHILDREN,SEX ASSLT OF CHILD BY FAM MBR,RESIDENCE,False,True,...,34.0,49.0,02,1176802.0,1835545.0,2011,02/10/2018 03:50:01 PM,41.704075,-87.628194,"(41.704075384, -87.628193565)"
1,10295795,HY483570,01/01/2011 12:00:00 AM,061XX N FAIRFIELD AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,50.0,2.0,11,1156892.0,1940584.0,2011,02/09/2018 03:44:29 PM,41.992738,-87.698256,"(41.992738481, -87.698256423)"
2,10308026,HY496701,01/01/2011 12:00:00 AM,056XX W GOODMAN ST,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,True,...,45.0,11.0,20,1137936.0,1932750.0,2011,02/10/2018 03:50:01 PM,41.971606,-87.768174,"(41.971605914, -87.768174187)"
3,11616799,JC177993,01/01/2011 12:00:00 AM,046XX S LA CROSSE AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,OTHER,False,False,...,22.0,56.0,11,NaN,NaN,2011,03/09/2019 04:05:42 PM,NaN,NaN,NaN
4,10902200,JA212148,01/01/2011 12:00:00 AM,0000X E 59TH ST,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,APARTMENT,False,False,...,20.0,40.0,11,NaN,NaN,2011,04/06/2017 03:56:56 PM,NaN,NaN,NaN


**.groupby()** is another powerful built-in function in Pandas. We can use it to group the entries of a DataFrame according to the values of the specific feature. Then, the following function The .groupby() method is applied to a DataFrame and groups it according to a feature. Then, we can apply some simple or more complicated functions on that grouped object. This is a very important tool for every data scientist working on tabular or structured data as it will help you to manipulate data easily and in a more effective way.

In [115]:
p = 0.2 #percentage missing data required

mask = np.random.choice([np.nan,1], size=len(df_data), p=[p,1-p])
df_datanan = df_data.copy()
df_datanan['Ward'] =  df_datanan['Ward'] * mask

In [116]:
missing_trans = lambda x: x.fillna(x.mean())
df_datanan_grouped = df_datanan.groupby('Primary Type')['Ward']
df_datanan_grouped.transform(missing_trans)

0        22.0
1         6.0
2        50.0
3        48.0
4        40.0
         ... 
36077    34.0
36078    27.0
36079     8.0
36080     2.0
36081    17.0
Name: Ward, Length: 36082, dtype: float64

In [ ]:

restaurant_grouped = restaurant.groupby('day')
filter_trans = lambda x : x['total_bill'].mean() > 20
restaurant_filtered = restaurant_grouped.filter(filter_trans)
     